# Определить уровень лояльности клиента

Заказчик исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос, необходимо подготовить дашборд с его итогами.

Для оценки определили три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS нужно расчитать по формуле: % «сторонников» - % «критиков».

<a id='content'></a>
**Оглавление**

**[Загрузка данных и подготовка их к анализу](#part_1)**
* [Импорт библиотек, загрузка данных](#part_1.1)
* [Проверка данных](#part_1.2)

**[Вывод](#part_2)**

**Презентация**
* Презентация в pdf формате: <>

<a id='part_1'></a>
## Загрузка данных и подготовка их к анализу

<a id='part_1.1'></a>
**Импорт библиотек, загрузка данных**  
[*Назад к оглавлению*](#content)

In [1]:
#Импортируем библиотеки
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
#Подключаемся к БД
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
#Запрос SQL
query = """
SELECT user.user_id,
        user.lt_day,
        user.age,
        user.gender_segment,
        user.os_name,
        user.cpe_type_name,
        location.country,
        location.city,
        age_segment.title as age_segment,
        traffic_segment.title as traffic_segment,
        lifetime_segment.title as lifetime_segment,
        user.nps_score 
FROM user
LEFT JOIN location ON user.location_id=location.location_id
LEFT JOIN age_segment ON user.age_gr_id=age_segment.age_gr_id
LEFT JOIN traffic_segment ON user.tr_gr_id=traffic_segment.tr_gr_id
LEFT JOIN lifetime_segment ON user.lt_gr_id=lifetime_segment.lt_gr_id;
"""

In [4]:
#Сохрнаим выгрузку в переменную и посмотрим на таблицу
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10


<a id='part_1.2'></a>
**Проверка данных**  
[*Назад к оглавлению*](#content)

In [5]:
#посмотрим общую информацию по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 12 columns):
user_id             502493 non-null object
lt_day              502493 non-null int64
age                 501939 non-null float64
gender_segment      501192 non-null float64
os_name             502493 non-null object
cpe_type_name       502493 non-null object
country             502493 non-null object
city                502493 non-null object
age_segment         502493 non-null object
traffic_segment     502493 non-null object
lifetime_segment    502493 non-null object
nps_score           502493 non-null int64
dtypes: float64(2), int64(2), object(8)
memory usage: 46.0+ MB


In [6]:
#посчитаем кол-во незаполненных данных
(df['gender_segment'].isna().sum() + df['age'].isna().sum()) / df['user_id'].count()

0.003691593713743276

В двух солбцах есть незаполненные ячейки. Так как строк мало удалим их

In [7]:
#удалим строки с незаполненными данными
df.dropna(subset=['gender_segment'], inplace=True)
df.dropna(subset=['age'], inplace=True)

In [8]:
#проверим дубликаты
df.duplicated().sum()

0

In [9]:
#проверим дубликаты по идентификатору клиента
df['user_id'].duplicated().sum()

0

In [10]:
#функция категоризации 
def is_new(x):
    if x < 365:
        return 'new'
    return 'old'

def nps_group(y):
    if y <= 6:
        return 'detractors'
    if y <= 8:
        return 'passives'
    return 'promoters'

def gender(z):
    if z == 1:
        return 'женщина'
    return 'мужчина'

In [11]:
#зальем датсет через apply и функцию
df['is_new'] = df['lt_day'].apply(is_new)
df['nps_group'] = df['nps_score'].apply(nps_group)
df['gender_segment'] = df['gender_segment'].apply(gender)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,is_new,nps_group
0,A001A2,2320,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,old,promoters
1,A001WF,2344,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,old,promoters
2,A003Q7,467,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,old,promoters


In [12]:
df['age_segment'] = df['age_segment'].str.split().str[1]
df['traffic_segment'] = df['traffic_segment'].str.split().str[1]
df['lifetime_segment'] = df['lifetime_segment'].str.split().str[1]
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,is_new,nps_group
0,A001A2,2320,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,old,promoters
1,A001WF,2344,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,old,promoters
2,A003Q7,467,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,old,promoters


In [13]:
#посмотрим общую информацию по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501152 entries, 0 to 502492
Data columns (total 14 columns):
user_id             501152 non-null object
lt_day              501152 non-null int64
age                 501152 non-null float64
gender_segment      501152 non-null object
os_name             501152 non-null object
cpe_type_name       501152 non-null object
country             501152 non-null object
city                501152 non-null object
age_segment         501152 non-null object
traffic_segment     501152 non-null object
lifetime_segment    501152 non-null object
nps_score           501152 non-null int64
is_new              501152 non-null object
nps_group           501152 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 57.4+ MB


<a id='part_2'></a>
## Вывод
[*Назад к оглавлению*](#content)

В файле 12 столбцов, столбцы содержат данные о:  
***user_id*** — Идентификатор клиента;  
***lt_day*** — Количество дней «жизни» клиента;  
***age*** — Возраст;  
***gender_segment*** — Пол;  
***os_name*** — Тип операционной системы;  
***cpe_type_name*** — Тип устройства;  
***country*** — Страна проживания;  
***city*** — Город проживания;  
***age_segment*** — Возрастной сегмент;  
***traffic_segment*** — Сегмент по объёму потребляемого трафика;  
***lifetime_segment*** — Сегмент по количеству дней «жизни»;  
***nps_score*** — Оценка клиента в NPS-опросе;  

Для анализа добавили 2 столбца: 
***is_new*** — Поле хранит информацию о том, является ли клиент новым (новые клиенты, если количество дней «жизни» составляет не более 365 дней и столбец);  
***nps_group*** — Поле хранит информацию о том, к какой группе относится оценка клиента в опросе.  

Для удобства преобразовали столбцы ***gender_segment***, ***age_segment***, ***traffic_segment***, ***lifetime_segment***.  
Удалили строки с незаполнеными данными.

In [14]:
#вывод файла
df.to_csv('file.csv', index=False, sep=';')